In [36]:
# Wide Receivers with at least 5 targets per game for each week with defined year range 

In [39]:
# Import the necessary modules 
import nfl_data_py as nfl
import pandas as pd
from IPython.display import display, HTML

# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Define the years of interest
years = list(range(2017, 2023 + 1))

# Base columns for all positions
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name',
    'position', 'position_group', 'recent_team', 'opponent_team',
    'fantasy_points', 'fantasy_points_ppr'
]

# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

# Define the WR-specific target-related columns
wr_target_columns = ['targets', 'target_share', 'receptions', 'receiving_yards']

# Combine base columns with WR-specific columns
wr_all_columns = base_columns + wr_columns
wr_tgt_columns = base_columns + wr_target_columns

# Import data for all the specified years
nfl_data_all_years = nfl.import_weekly_data(
    years=years,
    columns=wr_all_columns
)

# Filter to show only WR positions
wr_data_all_years = nfl_data_all_years[nfl_data_all_years['position'] == 'WR']

# Select only the relevant columns for WRs
wr_tgt_data_all_years = wr_data_all_years[wr_tgt_columns]

# Filter WRs with at least 5 targets per game for each week
wr_with_min_5_targets_all_years = wr_tgt_data_all_years[wr_tgt_data_all_years['targets'] >= 5]

# Sort by year, week, and then by targets for better analysis
wr_with_min_5_targets_sorted_all_years = wr_with_min_5_targets_all_years.sort_values(by=['season', 'week', 'targets'], ascending=[False, True, False])

# Remove the 'player_id' and 'position_group' columns
wr_filtered_data_all_years = wr_with_min_5_targets_sorted_all_years.drop(columns=['player_id', 'position_group'])

# Format decimals to two decimal places for all columns
wr_filtered_data_all_years = wr_filtered_data_all_years.round(2)

# Specifically format 'fantasy_points' and 'fantasy_points_ppr' as strings with two decimal places
wr_filtered_data_all_years['fantasy_points'] = wr_filtered_data_all_years['fantasy_points'].apply(lambda x: f"{x:.2f}")
wr_filtered_data_all_years['fantasy_points_ppr'] = wr_filtered_data_all_years['fantasy_points_ppr'].apply(lambda x: f"{x:.2f}")

# Display the refined data
display(HTML(wr_filtered_data_all_years.to_html(index=False)))


Downcasting floats.


season,season_type,week,player_name,position,recent_team,opponent_team,fantasy_points,fantasy_points_ppr,targets,target_share,receptions,receiving_yards
2023,REG,1,T.Hill,WR,MIA,LAC,33.50,44.50,15,0.34,11,215.0
2023,REG,1,P.Nacua,WR,LA,SEA,11.90,21.90,15,0.41,10,119.0
2023,REG,1,D.Hopkins,WR,TEN,NO,6.50,13.50,13,0.39,7,65.0
2023,REG,1,S.Diggs,WR,BUF,NYJ,16.20,26.20,13,0.33,10,102.0
2023,REG,1,J.Jefferson,WR,MIN,TB,15.00,24.00,12,0.27,9,150.0
2023,REG,1,K.Bourne,WR,NE,PHI,18.40,24.40,11,0.20,6,64.0
2023,REG,1,C.Ridley,WR,JAX,IND,16.10,24.10,11,0.34,8,101.0
2023,REG,1,M.Pittman,WR,IND,JAX,15.70,23.70,11,0.28,8,97.0
2023,REG,1,N.Collins,WR,HOU,BAL,8.00,14.00,11,0.26,6,80.0
2023,REG,1,R.Woods,WR,HOU,BAL,5.70,11.70,10,0.23,6,57.0
